In [291]:
#These are the libraries you can use.  You may add any libraries directy related to threading if this is a direction
#you wish to go (this is not from the course, so it's entirely on you if you wish to use threading).  Any
#further libraries you wish to use you must email me, james@uwaterloo.ca, for permission.

from IPython.display import display, Math, Latex

import pandas as pd
import numpy as np
import numpy_financial as npf
import yfinance as yf
import matplotlib.pyplot as plt
import random
from datetime import datetime
from scipy.optimize import minimize
import time ## REMOVE THIS LATER

## Group Assignment
### Team Number: 12
### Team Member Names: Sharuga, Derek, Alex
### Team Strategy Chosen: Market Meet

Disclose any use of AI for this assignment below (detail where and how you used it).  Please see the course outline for acceptable uses of AI.

CHECK DELIST:
- Purpose: checks that there is actually valid market data for this stock
- Code: if 'Close' not in data.columns or data['Close'].isnull().all():

PORTFOLIO GENERATOR
- Purpose: calculate how many shares of each stock to buy, and how much to pay in fees
- Code:

portfolio_value = 1000000 * weights  # Portfolio value based on optimized weights

shares = portfolio_val / np.array([ticker_prices[ticker] for ticker in selected_tickers])  # Number of shares

fee = np.minimum(3.95, 0.001 * shares)

portfolio_val -= fee


In [295]:
#October 01, 2023 to September 30, 2024, 2023

START_DATE = '2023-11-25'
END_DATE = '2024-11-23'

 # date on which we will take stock Close prices
TARGET_DATE = '2024-11-18'

# creating a $1000000 portfolio
INVESTMENT = 1000000

In [296]:
# filter out invalid stocks
def validity(tickers):

    # list to keep all of t
    # he valid stocks
    final_list = []

    for ticker in tickers:
        #only append the stock to the final stock list if it is isn't delisted, has a US or CAD currency, and has the required monthly share volume
        if (not check_delist(ticker) and
            check_currency(ticker) and
            check_volume(ticker)):
                final_list.append(ticker)
    return final_list

# checks if the consumed Ticker is delisted
def check_delist(ticker):
    stock = yf.Ticker(ticker)
    try:
        data = stock.history(period='1d')
        time.sleep(0.3)
        if data.empty:
            #if we can't find any data on the stock, it's delisted
            return True
        else:
            #check that there is actually valid market data for this stock
            if 'Close' not in data.columns or data['Close'].isnull().all():
                return True
            return False
    except Exception as e:
        #if there is an error in finding the stock's data, we can assume that it's delisted
        return True

# checks if the consumed Ticker meets the requirement of an average monthly share volume of 100,000 shares
def check_volume(ticker):
    volume = yf.Ticker(ticker).history(period='1d')['Volume']
    time.sleep(0.3)
    avg_monthly_volume = volume.resample('ME').mean()
    return avg_monthly_volume.mean() >= 100000

# checks whether or not the consumed Ticker is listed in CAD or in USD
def check_currency(ticker):
    stock = yf.Ticker(ticker)
    try:
        currency = stock.info.get('currency', None)
        return currency in ['USD', 'CAD']
    except Exception as e:
        return False  # if there's an error retrieving the data, we can assume it doesnt satisfy the required monthly volume

# read in the provided Ticker file
all_data = pd.read_csv('Tickers_Example.csv', header=None)

# create a new DataFrame only containing the valid Tickers in the consumed CSV
all_data = pd.DataFrame(validity(all_data[0]))
all_data.rename(columns = {0:'Ticker'}, inplace=True)

$AGN: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$CELG: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$MON: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")
$RTN: possibly delisted; no price data found  (period=1d) (Yahoo error = "No data found, symbol may be delisted")


In [297]:
# 
def get_sector(tickers):
    sectors = []

    for ticker in tickers:
        sectors.append(yf.Ticker(ticker).info.get('sector'))

    return sectors

In [298]:
# add a column for all stocks' respective tickers
all_data = all_data.assign(Sector=get_sector(all_data['Ticker']))

all_data

,Ticker,Sector
0,AAPL,Technology
1,ABBV,Healthcare
2,ABT,Healthcare
3,ACN,Technology
4,AIG,Financial Services
5,AMZN,Consumer Cyclical
6,AXP,Financial Services
7,BA,Industrials
8,BAC,Financial Services
9,BB.TO,Technology


In [299]:
# fetch TSX60 index information
tsx60_info = yf.Ticker('XIU.TO')
tsx60 = tsx60_info.history(start=START_DATE, end=END_DATE, interval='1d')
time.sleep(0.3)

# fetch SP500 index information
sp500_info = yf.Ticker('^GSPC')
sp500 = sp500_info.history(start=START_DATE, end=END_DATE, interval='1d')

# reindex sp500 so that it shares the same index as tsx60
sp500 = sp500.reindex(tsx60.index, method='nearest')

# calculate the average market return
market_returns = (tsx60['Close'].pct_change() + sp500['Close'].pct_change())/2
market_returns.index = pd.to_datetime(market_returns.index.strftime('%Y-%m-%d'))
market_returns = market_returns.dropna()
# market_returns_mean = market_returns.mean()

# store the average market returns in a DataFrame
market_returns_df = pd.DataFrame(market_returns)

# display(market_returns_df.index)

In [ ]:
# create a DataFrame to store 
ticker_returns = pd.DataFrame(columns=all_data['Ticker'])

# calculate and store all returns for all all Tickers
for ticker in all_data['Ticker']:
        data = yf.Ticker(ticker).history(start=START_DATE, end=END_DATE)['Close']
        time.sleep(0.3)
        ticker_returns[ticker] = data.pct_change().dropna()  # this code calculates daily returns
        
ticker_returns.index = pd.to_datetime(ticker_returns.index.strftime("%Y-%m-%d"))

ticker_returns

Ticker,AAPL,ABBV,ABT,ACN,AIG,AMZN,AXP,BA,BAC,BB.TO,...,QCOM,RY.TO,SHOP.TO,T.TO,TD.TO,TXN,UNH,UNP,UPS,USB
Date,,,,,,,,,,,,,,,,,,,,,
2023-11-28,0.003214,-0.007262,-0.006328,0.000391,-0.009349,-0.004738,0.009876,0.013999,-0.001015,-0.015810,...,-0.012559,-0.006857,-0.006765,-0.002076,-0.005537,0.001966,-0.005629,0.006277,0.007688,0.006795
2023-11-29,-0.005410,0.003042,0.015383,0.002345,0.002939,-0.004829,0.010685,0.009264,0.026414,0.028112,...,0.016854,0.012700,-0.001102,-0.007488,0.008353,0.001962,-0.010268,0.001085,0.010283,0.020248
2023-11-30,0.003063,0.028087,0.006369,-0.000600,0.015116,-0.001572,0.019949,0.032081,0.013967,-0.027344,...,0.008912,0.032068,-0.008724,0.017603,-0.006723,-0.003198,0.033627,0.017158,-0.004465,0.008732
2023-12-01,0.006791,0.007163,0.005657,0.014768,0.002431,0.006434,0.017509,0.009671,0.015415,0.040161,...,0.004804,0.002610,0.008801,0.028830,-0.010031,0.016371,-0.007131,0.027079,0.021766,0.033054
2023-12-04,-0.009465,0.005160,0.002956,-0.004822,0.000455,-0.014895,-0.002935,0.004276,-0.004522,0.017375,...,0.002159,0.002522,-0.015243,0.010008,0.004151,0.014239,0.002047,0.010935,0.006520,0.002793
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-15,-0.014109,-0.027354,0.007563,-0.023476,0.000264,-0.041942,-0.004960,0.014840,0.018518,-0.017699,...,-0.021162,-0.005347,-0.003650,0.002343,-0.010547,-0.023689,-0.001551,-0.000042,0.010781,-0.008149
2024-11-18,0.013422,0.007819,0.012597,-0.002206,0.007127,-0.004491,-0.004636,0.026250,-0.000642,0.012012,...,0.024922,0.002746,-0.028652,0.011220,0.000888,0.030131,-0.004356,-0.004924,0.020311,0.014228
2024-11-19,0.001140,0.001744,-0.001960,0.003288,-0.013235,0.014427,0.000035,0.012025,-0.006635,-0.029674,...,0.001277,0.000524,-0.016836,0.000925,-0.011284,-0.029491,-0.021453,-0.004778,-0.014283,-0.004149


In [319]:
common_dates = market_returns_df.index.intersection(ticker_returns.index)

market_returns_df = market_returns_df.loc[common_dates]
ticker_returns = ticker_returns.loc[common_dates]

months = market_returns_df.resample('MS').bfill().index

for month in months:
    temp = market_returns_df[(market_returns_df.index.month == month.month) & (market_returns_df.index.year == month.year)].index
    if len(temp) < 18:
        market_returns_df.drop(index=temp, inplace=True)
        ticker_returns.drop(index=temp, inplace=True)

print([val for val in ticker_returns.index if val not in market_returns_df.index])

print(len(market_returns_df))
print(len(ticker_returns))

[]
226
226


In [306]:
all_data['Ticker']

0        AAPL
1        ABBV
2         ABT
3         ACN
4         AIG
5        AMZN
6         AXP
7          BA
8         BAC
9       BB.TO
10       BIIB
11         BK
12        BLK
13        BMY
14          C
15        CAT
16         CL
17         KO
18        LLY
19        LMT
20         MO
21        MRK
22        PEP
23        PFE
24         PG
25         PM
26       PYPL
27       QCOM
28      RY.TO
29    SHOP.TO
30       T.TO
31      TD.TO
32        TXN
33        UNH
34        UNP
35        UPS
36        USB
Name: Ticker, dtype: object

In [307]:
portfolio_data = []
def portfolio_generator(tickers):
    global portfolio_data
   
    # this dictionary will store Ticker closing prices on the target date
    ticker_prices = {} 
     
    # populate ticker_prices
    for ticker in tickers:
        stock = yf.Ticker(ticker)
        history = stock.history(start=TARGET_DATE, end='2024-11-06')
        time.sleep(0.3)  # Add a small delay to avoid rate limiting
        
        # Fetch the closing price for the target date
        if not history.empty and TARGET_DATE in history.index.strftime('%Y-%m-%d'):
            close_price = history.loc[history.index.strftime('%Y-%m-%d') == TARGET_DATE, 'Close'].values[0]
            ticker_prices[ticker] = close_price
        else:
            ticker_prices[ticker] = None  # No data available for that date
    
    # creating portfolios -- starting wtih 12 stocks, going upto 24 stocks
    for i in range(12, 25):

        # for the ith portfolio, select the first i tickers
        selected_tickers = tickers[:i]
        
        # objective function -- will be used in optimization steps
        # returns the tracking error between the portfolio's returns and the market's returns
        def objective(weights):
            return np.std(np.dot(ticker_returns[selected_tickers].dropna().values, weights) - market_returns_df.dropna().values)
        
        # initial weights are calcualted so that, to start, every stock has equal weight
        initial_weights = [1 / len(selected_tickers)] * len(selected_tickers)
        
        # this constraint ensures that all of the stock weights sum to 1 (100%)
        def sum_weights(weights):
            return sum(weights) - 1
        
        constraints = [{'type': 'eq', 'fun': sum_weights}]
        
        # the bounds for each stock are that each stock must have a minimum weight of 1/2n% and maximum weight of 15%
        bounds = [(1 / (2 * len(selected_tickers)), 0.15)] * len(selected_tickers)
        
        # optimize the portfolio weights
        result = minimize(objective, initial_weights, constraints=constraints, bounds=bounds)
        
        if result.success:

            weights = result.x
                        
            # calculate portfolio value and the number of shares to purchase
            portfolio_value = 1000000 * weights
            shares = portfolio_val / np.array([ticker_prices[ticker] for ticker in selected_tickers]) 

            # calculate fees
            fee = np.minimum(3.95, 0.001 * shares)
            
            # subtract fees from portfolio value
            portfolio_val -= fee
            
            # Add to the results
            portfolio_data.append({
                'tickers': ', '.join(selected_tickers),  # Tickers in the portfolio
                'weights': result.x,  # Optimized weights
                'STD Between Stock and Market Index': result.fun,  # Standard deviation (risk)
                'num_stocks': i,  # Number of stocks in the portfolio
                'total_cost': portfolio_val.sum() + sum(fee),  # Total cost after fees
                'shares_purchased': shares,  # Shares purchased
                'portfolio_value': portfolio_val.sum(),  # Total portfolio value after fees
                'final_fees': sum(fee)
            })

    # Return the final portfolio data as a DataFrame
    return pd.DataFrame(portfolio_data)

# Example usage
  # Initialize the global portfolio data list
portfolio_df = portfolio_generator(all_data['Ticker'])  # Assuming all_data['Ticker'] is a list of tickers
portfolio_df


$AAPL: possibly delisted; no price data found  (1d 2024-11-18 -> 2024-11-06) (Yahoo error = "Invalid input - start date cannot be after end date. startDate = 1731906000, endDate = 1730869200")
$ABBV: possibly delisted; no price data found  (1d 2024-11-18 -> 2024-11-06) (Yahoo error = "Invalid input - start date cannot be after end date. startDate = 1731906000, endDate = 1730869200")
$ABT: possibly delisted; no price data found  (1d 2024-11-18 -> 2024-11-06) (Yahoo error = "Invalid input - start date cannot be after end date. startDate = 1731906000, endDate = 1730869200")
$ACN: possibly delisted; no price data found  (1d 2024-11-18 -> 2024-11-06) (Yahoo error = "Invalid input - start date cannot be after end date. startDate = 1731906000, endDate = 1730869200")
$AIG: possibly delisted; no price data found  (1d 2024-11-18 -> 2024-11-06) (Yahoo error = "Invalid input - start date cannot be after end date. startDate = 1731906000, endDate = 1730869200")
$AMZN: possibly delisted; no price dat

KeyboardInterrupt: 

## Contribution Declaration

The following team members made a meaningful contribution to this assignment:

Insert Names Here.